In [1]:
import numpy as np
from deck import full_euchre_deck
from branch_calc import filter_branch_by_hand
from dealer import dealer

In [2]:
game = dealer(deck=full_euchre_deck ,players=4)
game.deal_cards()
hands = [game.hand1, game.hand2, game.hand3, game.hand4]


In [3]:
game.hand1

array([[ 13,   0],
       [  9,   0],
       [-12,   0],
       [ 12,   0],
       [  0,  11]])

In [4]:
# all playable hands
grids = np.meshgrid(*[np.arange(len(a)) for a in hands], indexing='ij')
product_indices = np.stack(grids, axis=-1).reshape(-1, len(hands))

# Gather the cross product result based on indices, 625 results total
cross_product = np.array([[hands[i][idx] for i, idx in enumerate(row)] for row in product_indices])

In [5]:
target = np.array(game.hand1[0])
contains_target = np.any(np.all(cross_product == target, axis=-1), axis=1)

# Use the boolean mask to return only rows containing [0, 10]
branch1 = cross_product[contains_target]
game.hand2, game.hand3, game.hand4

(array([[  0,  13],
        [  0,  14],
        [ -9,   0],
        [  0, -10],
        [  0,  -9]]),
 array([[-10,   0],
        [-11,   0],
        [  0, -14],
        [-13,   0],
        [  0, -12]]),
 array([[ 14,   0],
        [  0, -13],
        [ 10,   0],
        [  0, -11],
        [  0,  12]]))

In [ ]:
branch1 = filter_branch_by_hand(branch1, game.hand2, 1, target)
branch1 = filter_branch_by_hand(branch1, game.hand3, 2, target)
branch1 = filter_branch_by_hand(branch1, game.hand4, 3, target)
len(branch1), branch1

(50,
 array([[[ 13,   0],
         [  0,  13],
         [-10,   0],
         [ 14,   0]],
 
        [[ 13,   0],
         [  0,  13],
         [-10,   0],
         [ 10,   0]],
 
        [[ 13,   0],
         [  0,  13],
         [-11,   0],
         [ 14,   0]],
 
        [[ 13,   0],
         [  0,  13],
         [-11,   0],
         [ 10,   0]],
 
        [[ 13,   0],
         [  0,  13],
         [  0, -14],
         [ 14,   0]],
 
        [[ 13,   0],
         [  0,  13],
         [  0, -14],
         [ 10,   0]],
 
        [[ 13,   0],
         [  0,  13],
         [-13,   0],
         [ 14,   0]],
 
        [[ 13,   0],
         [  0,  13],
         [-13,   0],
         [ 10,   0]],
 
        [[ 13,   0],
         [  0,  13],
         [  0, -12],
         [ 14,   0]],
 
        [[ 13,   0],
         [  0,  13],
         [  0, -12],
         [ 10,   0]],
 
        [[ 13,   0],
         [  0,  14],
         [-10,   0],
         [ 14,   0]],
 
        [[ 13,   0],
         [  0,  1